Isak Schwartz: 6 hours
Anton Forsberg: 6 hours
# Assignment 4
## notes
naive bayes
train given spam, ham


In [1]:
#Import scikit-learn dataset library
from sklearn import datasets

# Import train_test_split function
from sklearn.model_selection import train_test_split

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

import numpy as np

import pandas as pd

import os

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.naive_bayes import BernoulliNB

## 1: Preprocessing

In [2]:
def createTokens(pathsAndTargets): # crates a list of tokens for all files in a given directory   
    files = []
    target = []
    for pathAndTarget in pathsAndTargets:
        
        for file in os.listdir(pathAndTarget[0]):
            filePath = (pathAndTarget[0] + "/" ) + file
            try:
                unFilteredFile = open(filePath,'r', encoding="utf-8").read()
                filteredFile = filetrFile(unFilteredFile)
                files.append(filteredFile)
                target.append(pathAndTarget[1])
            except UnicodeDecodeError:
                pass
            
    vectorizer = CountVectorizer(analyzer='word')
    # make vector from documents
    dataset = vectorizer.fit_transform(files).toarray()
    filterList = filterWords(vectorizer.get_feature_names(), dataset)
    print(filterList)
    vectorizer2 = CountVectorizer(stop_words = frozenset(filterList))
    vectorizer2._validate_vocabulary()
    dataset = vectorizer2.fit_transform(files).toarray()
    print(vectorizer2.get_feature_names())
    dataset = vectorizer.fit_transform(files).toarray()
    #print(vectorizer.get_feature_names())
    #print word count table
    #wordTable(vectorizer.get_feature_names(), dataset)
    
    return (dataset, target)

In [3]:
def printBayesianAccuracies(pathsAndTargets):
    dataset = createTokens(pathsAndTargets)
    # Split dataset into training set and test set (70-30)
    X_train, X_test, y_train, y_test = train_test_split(dataset[0], dataset[1], test_size=0.3,random_state=0)#,random_state=109)
    
    #Create a Naïve Bayes Classifiers
    classifiers = {(MultinomialNB(),"Multinomial naive bayes"), (BernoulliNB(binarize = 0), "Bernoulli naive bayes")}
    
    for classifier in classifiers:
        dataset = classifier[0]
        
        #Train the model using the training sets
        dataset.fit(X_train, y_train)

        #Predict the response for test dataset
        y_pred = dataset.predict(X_test)

        # Model Accuracy, how often is the classifier correct?
        print("Accuracy of " + classifier[1] + ": ",metrics.accuracy_score(y_test, y_pred))

In [4]:
def wordTable(words, occurences):
    # give length equal to the data entries
    totalOccurences = np.zeros(len(occurences[0]))
    for mail in occurences:
        for i in range (len(mail)):
            totalOccurences[i] += mail[i]
    # make table
    d = {'word': words, 'occurences': totalOccurences}
    df = pd.DataFrame(data=d)
    df.sort_values(by=['occurences'], inplace = True)
    print(df)

In [5]:
def filterWords(words, occurences):
    # give length equal to the data entries
    totalOccurences = np.zeros(len(occurences[0]))
    for mail in occurences:
        for i in range (len(mail)):
            totalOccurences[i] += mail[i]
    # make filterList
    d = {'word': words, 'occurences': totalOccurences}
    df = pd.DataFrame(data=d)
    unfrequent = df.loc[df['occurences'] < 10]['word'].values
    frequent = df.loc[df['occurences'] > 1000]['word'].values
    print(type(unfrequent))
    return np.concatenate((unfrequent , frequent))

In [6]:
def filetrFile(file):
    try: 
        index = file.index("<!-- ### footer ### -->")
        file = file[:index]
    except ValueError:
        pass
    try: 
        index = file.index("<!-- ### \header ### -->")+ len("<!-- ### \header ### -->")
        file = file[index:]
    except ValueError:
        pass

    try:
        index = file.index("\n\n")
        file = file[index:]
    except ValueError:
        pass

    try:
        index = file.index("<html>")+ len("<html>")
        file = file[index:]
    except ValueError:
        pass
    
    return file


### 1.a : "run" entire text, including header and footer

In [7]:
easyPaths = {("spam_2",0),("easy_ham_2",1)}
hardPaths = {("spam_2",0),("hard_ham",1)}
    
print("spam_2 and easy_ham_2 Accuracies:")
printBayesianAccuracies(easyPaths)
print("spam_2 and hard_ham Accuracies:")
printBayesianAccuracies(hardPaths)

paths = {("spam_2",0),("easy_ham_2",1)}

dataset = createTokens(paths) 

spam_2 and easy_ham_2 Accuracies:
<class 'numpy.ndarray'>
['00000000' '0000000000' '00000000000000000' ... 'www' 'you' 'your']
['00', '0000', '00000', '000033', '000066', '000080', '000099', '0000a0', '0000cc', '0000ff', '0001pt', '001', '00125e', '0022', '0025', '0030a0', '003366', '003399', '0033ff', '0066', '006600', '006699', '0066cc', '0066ff', '008000', '008080', '0099ff', '00am', '00ff00', '00ff99', '00ffff', '01', '0100', '02', '0200', '02131', '03', '04', '0400', '043bc0', '05', '0500', '0502', '06', '0600', '07', '0700', '0773', '08', '08039', '08296b', '09', '0cm', '0e', '0in', '0pt', '0px', '1000', '101', '1010110111011110101010101110101010101010101011010101010101010101101011010101010101010101010110101010101010101100010110111010101010100011010101101010101101', '102', '1024', '103', '104', '105', '105th', '106', '107', '108', '1087', '109', '1095548', '10pt', '10px', '11', '110', '1100', '110110110110101010111010101010101010011010101010101010101011010101101101010101010101010

Accuracy of Bernoulli naive bayes:  0.9140625
Accuracy of Multinomial naive bayes:  0.9166666666666666
spam_2 and hard_ham Accuracies:
<class 'numpy.ndarray'>
['0000000000' '00000000000000000' '000000000000000000000' ... 'you' 'your'
 'zdnet']
['00', '0000', '00000', '000033', '000066', '000080', '000099', '0000a0', '0000cc', '0000ff', '0001pt', '000623', '000625', '000626', '000627', '000628', '001', '00125e', '0022', '0025', '0030a0', '003366', '00339', '003399', '0033ff', '0066', '006600', '006633', '006699', '0066cc', '0066ff', '008000', '008080', '009933', '0099ff', '00ff00', '00ff99', '00ffff', '01', '02', '021502_pointcamera', '03', '04', '0400', '043bc0', '05', '0502', '06', '07', '0773', '08', '08296b', '0966103254', '09document', '0a', '0b', '0c', '0cm', '0e', '0f', '0h', '0i', '0in', '0mm', '0pt', '0px', '0v', '0x015a4e35', '0xfffffff', '0xffffffff', '1000', '10000', '101', '10101101110111101010101011101010101010101010110101010101010101011010110101010101010101010101101010101

Accuracy of Bernoulli naive bayes:  0.9266666666666666
Accuracy of Multinomial naive bayes:  0.9555555555555556
<class 'numpy.ndarray'>
['00000000' '0000000000' '00000000000000000' ... 'www' 'you' 'your']
['00', '0000', '00000', '000033', '000066', '000080', '000099', '0000a0', '0000cc', '0000ff', '0001pt', '001', '00125e', '0022', '0025', '0030a0', '003366', '003399', '0033ff', '0066', '006600', '006699', '0066cc', '0066ff', '008000', '008080', '0099ff', '00am', '00ff00', '00ff99', '00ffff', '01', '0100', '02', '0200', '02131', '03', '04', '0400', '043bc0', '05', '0500', '0502', '06', '0600', '07', '0700', '0773', '08', '08039', '08296b', '09', '0cm', '0e', '0in', '0pt', '0px', '1000', '101', '1010110111011110101010101110101010101010101011010101010101010101101011010101010101010101010110101010101010101100010110111010101010100011010101101010101101', '102', '1024', '103', '104', '105', '105th', '106', '107', '108', '1087', '109', '1095548', '10pt', '10px', '11', '110', '1100', '110110110

### 1.b : split spam and ham in training / test sets

## 2 : python program

### 2.a : use 4 sets hamtrain, spamtrain, hamtest, spamtest

### 2.b : use naive bayes, classify test sets, report % ham, spam

## 5

list